# DataFrame Operations

Let's get some quick practice with your new Spark DataFrame skills, you will be asked some basic questions about some stock market data, in this case Walmart Stock from the years 2012-2017. This exercise will just ask a bunch of questions, unlike the future machine learning exercises, which will be a little looser and be in the form of "Consulting Projects", but more on that later!

For now, just answer the questions and complete the tasks below.

In [1]:
## To display notebook cell with horizontal scroll bar
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

#### Use the walmart_stock.csv file to Answer and complete the  tasks below!

#### Start a simple Spark Session

In [2]:
import findspark 
import numpy 
import pandas
findspark.init()
import pyspark
from  pyspark.sql import SparkSession 
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [3]:
from pyspark.sql.functions import col

In [4]:
import pyspark.sql.functions as F # for aggregation funs || min ,max ...

#### Load the Walmart Stock CSV File, have Spark infer the data types.

In [5]:
data = spark.read.csv('F:\courses\SUMMER_2023\ITI\PySpark\Day_2\walmart_stock.csv',header=True,inferSchema=True,samplingRatio=0.1)

#### What are the column names?

In [6]:
data.columns

['Date', 'Open', 'High', 'Low', 'Close', 'Volume', 'Adj Close']

#### What does the Schema look like?

In [7]:
data.printSchema()

root
 |-- Date: date (nullable = true)
 |-- Open: double (nullable = true)
 |-- High: double (nullable = true)
 |-- Low: double (nullable = true)
 |-- Close: double (nullable = true)
 |-- Volume: integer (nullable = true)
 |-- Adj Close: double (nullable = true)



#### Print out the first 5 columns.

In [8]:
data.show(5)

+----------+------------------+---------+---------+------------------+--------+------------------+
|      Date|              Open|     High|      Low|             Close|  Volume|         Adj Close|
+----------+------------------+---------+---------+------------------+--------+------------------+
|2012-01-03|         59.970001|61.060001|59.869999|         60.330002|12668800|52.619234999999996|
|2012-01-04|60.209998999999996|60.349998|59.470001|59.709998999999996| 9593300|         52.078475|
|2012-01-05|         59.349998|59.619999|58.369999|         59.419998|12768200|         51.825539|
|2012-01-06|         59.419998|59.450001|58.869999|              59.0| 8069400|          51.45922|
|2012-01-09|         59.029999|59.549999|58.919998|             59.18| 6679300|51.616215000000004|
+----------+------------------+---------+---------+------------------+--------+------------------+
only showing top 5 rows



#### Use describe() to learn about the DataFrame.

In [9]:
data.describe().show()

+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|summary|              Open|             High|              Low|            Close|           Volume|        Adj Close|
+-------+------------------+-----------------+-----------------+-----------------+-----------------+-----------------+
|  count|              1258|             1258|             1258|             1258|             1258|             1258|
|   mean| 72.35785375357709|72.83938807631165| 71.9186009594594|72.38844998012726|8222093.481717011|67.23883848728146|
| stddev|  6.76809024470826|6.768186808159218|6.744075756255496|6.756859163732991|  4519780.8431556|6.722609449996857|
|    min|56.389998999999996|        57.060001|        56.299999|        56.419998|          2094900|        50.363689|
|    max|         90.800003|        90.970001|            89.25|        90.470001|         80898100|84.91421600000001|
+-------+------------------+-----------------+--

## Bonus Question!
#### There are too many decimal places for mean and stddev in the describe() dataframe. Format the numbers to just show up to two decimal places. Pay careful attention to the datatypes that .describe() returns, we didn't cover how to do this exact formatting, but we covered something very similar. 

If you get stuck on this, don't worry, just view the solutions.

In [10]:
# Notice all data types are Strings! 
df.describe().printSchema()

NameError: name 'df' is not defined

In [ ]:
from pyspark.sql.functions import format_number

In [ ]:
result = data.describe()
result.select(result['summary'],
              format_number(result['Open'].cast('float'),2).alias('Open'),
              format_number(result['High'].cast('float'),2).alias('High'),
              format_number(result['Low'].cast('float'),2).alias('Low'),
              format_number(result['Close'].cast('float'),2).alias('Close'),
              result['Volume'].cast('int').alias('Volume')
             ).show()

+-------+--------+--------+--------+--------+--------+
|summary|    Open|    High|     Low|   Close|  Volume|
+-------+--------+--------+--------+--------+--------+
|  count|1,258.00|1,258.00|1,258.00|1,258.00|    1258|
|   mean|   72.36|   72.84|   71.92|   72.39| 8222093|
| stddev|    6.77|    6.77|    6.74|    6.76| 4519780|
|    min|   56.39|   57.06|   56.30|   56.42| 2094900|
|    max|   90.80|   90.97|   89.25|   90.47|80898100|
+-------+--------+--------+--------+--------+--------+



#### ################################################################## End of Bonus Question

#### Create a new dataframe with a column called HV Ratio that is the ratio of the High Price versus volume of stock traded for a day.

In [ ]:
#HV_Ratio =data['high']/data['volume']
HV_Ratio = data.select((data['high'] / data['volume']).alias('HV Ratio'))
HV_Ratio.show()

+--------------------+
|            HV Ratio|
+--------------------+
|4.819714653321546E-6|
|6.290848613094555E-6|
|4.669412994783916E-6|
|7.367338463826307E-6|
|8.915604778943901E-6|
|8.644477436914568E-6|
|9.351828421515645E-6|
| 8.29141562102703E-6|
|7.712212102001476E-6|
|7.071764823529412E-6|
|1.015495466386981E-5|
|6.576354146362592...|
| 5.90145296180676E-6|
|8.547679455011844E-6|
|8.420709512685392E-6|
|1.041448341728929...|
|8.316075414862431E-6|
|9.721183814992126E-6|
|8.029436027707578E-6|
|6.307432259386365E-6|
+--------------------+
only showing top 20 rows



#### What day had the Peak High in Price?

In [17]:
max_value = data.agg(F.max(data.High)).collect()[0][0]
max_value

90.970001

In [18]:
data.select("Date").where(data.High == max_value ).show()

+----------+
|      Date|
+----------+
|2015-01-13|
+----------+



#### What is the mean of the Close column?

In [ ]:
data.agg(F.avg(data.Close)).show()


+-----------------+
|       avg(Close)|
+-----------------+
|72.38844998012726|
+-----------------+



#### What is the max and min of the Volume column?

In [ ]:
data.agg(F.min(data.Volume),F.max(data.Volume)).show()


+-----------+-----------+
|min(Volume)|max(Volume)|
+-----------+-----------+
|    2094900|   80898100|
+-----------+-----------+



#### How many days was the Close lower than 60 dollars?

In [ ]:
data.select('Date').where(data.Close<60).count()

81

#### What percentage of the time was the High greater than 80 dollars ?
#### In other words, (Number of Days High>80)/(Total Days in the dataset)

In [ ]:
data.select("Date").count() 

1258

In [ ]:
data.select('Date').where(data.High>80).count()

115

In [ ]:
data.select('Date').where(data.High>80).count() / data.select("Date").count()  *100

9.141494435612083

#### What is the Pearson correlation between High and Volume?

In [ ]:
data.corr('High','Volume', method ='pearson')

-0.3384326061737161

#### u can use pyspark.ml.stat.Correlation
The main advantage of using pyspark.ml.stat.Correlation is its flexibility and the ability to handle missing values, as it offers different methods for correlation calculation and can automatically handle null values in the input dataframe. However, if you just need to calculate the Pearson correlation coefficient between two columns without additional features or missing value handling, using the corr function from pyspark.sql.functions is a more concise and straightforward solution.

+-------------------+
| corr(High, Volume)|
+-------------------+
|-0.3384326061737161|
+-------------------+



#### What is the max High per year?

In [ ]:
# first split date column year month day 
from pyspark.sql.functions import year, month, dayofmonth

# Assuming that your Spark dataframe is called "df" and the date column is named "date_col"
data = data.withColumn("year_col", year("Date"))
data = data.withColumn("month_col", month("Date"))
data = data.withColumn("day_col", dayofmonth("Date"))
data.show()

+----------+------------------+------------------+------------------+------------------+--------+------------------+--------+---------+-------+
|      Date|              Open|              High|               Low|             Close|  Volume|         Adj Close|year_col|month_col|day_col|
+----------+------------------+------------------+------------------+------------------+--------+------------------+--------+---------+-------+
|2012-01-03|         59.970001|         61.060001|         59.869999|         60.330002|12668800|52.619234999999996|    2012|        1|      3|
|2012-01-04|60.209998999999996|         60.349998|         59.470001|59.709998999999996| 9593300|         52.078475|    2012|        1|      4|
|2012-01-05|         59.349998|         59.619999|         58.369999|         59.419998|12768200|         51.825539|    2012|        1|      5|
|2012-01-06|         59.419998|         59.450001|         58.869999|              59.0| 8069400|          51.45922|    2012|        1| 

In [ ]:
data.groupBy("year_col").agg(F.max('High')).show()


+--------+---------+
|year_col|max(High)|
+--------+---------+
|    2015|90.970001|
|    2013|81.370003|
|    2014|88.089996|
|    2012|77.599998|
|    2016|75.190002|
+--------+---------+



#### What is the average Close for each Calendar Month?
#### In other words, across all the years, what is the average Close price for Jan,Feb, Mar, etc... Your result will have a value for each of these months. 

In [ ]:
data.groupBy("month_col").agg(F.avg('Close')).sort("month_col").show()


+---------+-----------------+
|month_col|       avg(Close)|
+---------+-----------------+
|        1|71.44801958415842|
|        2|  71.306804443299|
|        3|71.77794377570092|
|        4|72.97361900952382|
|        5|72.30971688679247|
|        6| 72.4953774245283|
|        7|74.43971943925233|
|        8|73.02981855454546|
|        9|72.18411785294116|
|       10|71.57854545454543|
|       11| 72.1110893069307|
|       12|72.84792478301885|
+---------+-----------------+



## Task 2


### Read "test1" dataset:

In [ ]:
test_data = spark.read.csv("test1.csv",header=True ,inferSchema=True,samplingRatio=0.1)

In [ ]:
test_data.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



### Display Salary of the people less than or equal to 20000

In [ ]:
test_data.where(col('Salary') <= 20000).show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



### Display Salary of the people less than or equal to 20000 and greater than or equal 15000

In [ ]:
test_data.where((col('Salary') <= 20000) & (col('Salary') >= 15000) ).show()

+-------+---+----------+------+
|   Name|age|Experience|Salary|
+-------+---+----------+------+
|  Sunny| 29|         4| 20000|
|   Paul| 24|         3| 20000|
| Harsha| 21|         1| 15000|
|Shubham| 23|         2| 18000|
+-------+---+----------+------+



## Task 3 

### Read "test3" dataset:

In [ ]:
data_test3 = spark.read.csv("test3.csv",header=True, inferSchema=True)

### Display dataset

In [ ]:
data_test3.show()

+---------+------------+------+
|     Name| Departments|salary|
+---------+------------+------+
|    Krish|Data Science| 10000|
|    Krish|         IOT|  5000|
|   Mahesh|    Big Data|  4000|
|    Krish|    Big Data|  4000|
|   Mahesh|Data Science|  3000|
|Sudhanshu|Data Science| 20000|
|Sudhanshu|         IOT| 10000|
|Sudhanshu|    Big Data|  5000|
|    Sunny|Data Science| 10000|
|    Sunny|    Big Data|  2000|
+---------+------------+------+



### Display schema

In [ ]:
data_test3.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Departments: string (nullable = true)
 |-- salary: integer (nullable = true)



### Group by "Name" column and using sum function on "Name" column

In [ ]:
data_test3.groupBy("Name").sum().show()

+---------+-----------+
|     Name|sum(salary)|
+---------+-----------+
|Sudhanshu|      35000|
|    Sunny|      12000|
|    Krish|      19000|
|   Mahesh|       7000|
+---------+-----------+



### Group by "Name" column and using avg function on "Name" column

In [ ]:
data_test3.groupBy("Name").avg().show()

+---------+------------------+
|     Name|       avg(salary)|
+---------+------------------+
|Sudhanshu|11666.666666666666|
|    Sunny|            6000.0|
|    Krish| 6333.333333333333|
|   Mahesh|            3500.0|
+---------+------------------+



### Group by "Departments" column and using sum function on "Departments" column

In [ ]:
data_test3.groupBy("Departments").sum().show()

+------------+-----------+
| Departments|sum(salary)|
+------------+-----------+
|         IOT|      15000|
|    Big Data|      15000|
|Data Science|      43000|
+------------+-----------+



### Group by "Departments" column and using mean function on "Departments" column:

In [ ]:
data_test3.groupBy("Departments").mean().show()

+------------+-----------+
| Departments|avg(salary)|
+------------+-----------+
|         IOT|     7500.0|
|    Big Data|     3750.0|
|Data Science|    10750.0|
+------------+-----------+



Group by "Departments" column and using count function on "Departments" column:

In [ ]:
data_test3.groupBy("Departments").count().show()

+------------+-----+
| Departments|count|
+------------+-----+
|         IOT|    2|
|    Big Data|    4|
|Data Science|    4|
+------------+-----+



### Apply agg to using sum function get the total of salaries

In [ ]:
data_test3.agg(F.sum('Salary')).show()

+-----------+
|sum(Salary)|
+-----------+
|      73000|
+-----------+

